In [1]:
import tensorflow as tf, tensorflow_addons as tfa
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dropout
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image
import numpy as np
import os
from PIL import Image, ImageOps
import cv2
from tqdm import tqdm

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
%%time

class FixedDropout(Dropout):
    def _get_noise_shape(self, inputs):
        if self.noise_shape is None:
            return self.noise_shape

        symbolic_shape = backend.shape(inputs)
        noise_shape = [symbolic_shape[axis] if shape is None else shape
                        for axis, shape in enumerate(self.noise_shape)]
        return tuple(noise_shape)
    
def dice_coef(y_true, y_pred, smooth=1.):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (
                K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def jaccard_coef(y_true, y_pred):
    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true + y_pred)
    jac = (intersection + 1.) / (union - intersection + 1.)
    return K.mean(jac)

def jaccard_distance(y_true, y_pred, smooth=100):
    """ Calculates mean of Jaccard distance as a loss function """
    intersection = tf.reduce_sum(y_true * y_pred, axis=(1,2))
    sum_ = tf.reduce_sum(y_true + y_pred, axis=(1,2))
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    jd =  (1 - jac) * smooth
    return tf.reduce_mean(jd)
learning_rate = 0.0001
weight_decay = 0.000001

optimizer = tfa.optimizers.AdamW(learning_rate=learning_rate, 
                                 weight_decay=weight_decay)

paths = [r'D:\models\crimea\models2\models\model_door_Unet_512_11.h5', 
         r'D:\models\crimea\models2\models\model_door_512_0.9108.h5', 
         r'D:\models\crimea\models2\models\model_wall_Unet_512_43.h5',
         r'D:\models\crimea\models2\models\model_wall_512_22.h5',
         r'D:\models\crimea\models2\models\model_window_512_0.8811.h5']

models = [load_model(name, custom_objects={'jaccard_distance':jaccard_distance,
                                            'jaccard_coef':jaccard_coef,
                                            'dice_coef':dice_coef,
                                            'FixedDropout':FixedDropout(rate=0.2)}) 
          for name in paths]
models

CPU times: total: 38.1 s
Wall time: 37.5 s


In [4]:
from PIL import Image, ImageEnhance
        
path = r'F:\Crimea\_test_'
names = os.listdir(path)
mask_door = np.zeros((512, 512, 3))
mask_wall = np.zeros((512, 512, 3))
mask_window = np.zeros((512, 512, 3))
D = 3

factor = 1.5

for name in tqdm(names[0:]):    
    size = Image.open(os.path.join(path, name)).size
    img_path = os.path.join(path, name)
    img = Image.open(img_path).resize((512, 512))
    if len(img.size) == 2:
        img = img.convert('RGB')
    enhancer = ImageEnhance.Sharpness(img)
    img = enhancer.enhance(factor)

    img_tensor = np.array(img, dtype=np.float64)
    img_tensor = np.expand_dims(img_tensor, axis=0)

    img_tensor /= 255.
    
    pred_door_unet = models[0].predict(img_tensor)[0]
    pred_door_unet = np.uint8(pred_door_unet * 255)
    pred_door_unet[pred_door_unet > 128] = 80
    pred_door_unet[pred_door_unet != 80] = 0
    
    pred_door_pspnet = models[1].predict(img_tensor)[0]
    pred_door_pspnet = np.uint8(pred_door_pspnet * 255)
    pred_door_pspnet[pred_door_pspnet > 128] = 80
    pred_door_pspnet[pred_door_pspnet != 80] = 0
    
    pred_door = cv2.bitwise_and(pred_door_unet, pred_door_pspnet)
    
    mask_door[:,:,0] = pred_door[:,:,]
    mask_door[:,:,1] = pred_door[:,:,]
    mask_door[:,:,2] = pred_door[:,:,]

        
    pred_wall_unet = models[2].predict(img_tensor)[0]
    pred_wall_unet = np.uint8(pred_wall_unet * 255)
    pred_wall_unet[pred_wall_unet > 128] = 255
    pred_wall_unet[pred_wall_unet < 255] = 0
    
    pred_wall_pspnet = models[3].predict(img_tensor)[0]
    pred_wall_pspnet = np.uint8(pred_wall_pspnet * 255)
    pred_wall_pspnet[pred_wall_pspnet > 128] = 255
    pred_wall_pspnet[pred_wall_pspnet < 255] = 0
    
    pred_wall = cv2.bitwise_and(pred_wall_unet, pred_wall_pspnet)
    
    mask_wall[:,:,0] = pred_wall
    mask_wall[:,:,1] = pred_wall
    mask_wall[:,:,2] = pred_wall
    

    pred_window = models[4].predict(img_tensor)[0]
    pred_window = cv2.resize(pred_window, (512, 512))
    pred_window = np.uint8(pred_window * 255)
    pred_window[pred_window > 128] = 160
    pred_window[pred_window != 160] = 0
    
    
    mask_window[:,:,0] = pred_window
    mask_window[:,:,1] = pred_window
    mask_window[:,:,2] = pred_window

    mask_door = cv2.resize(mask_door, (512, 512))
    mask_wall = cv2.resize(mask_wall, (512, 512))
    mask_window = cv2.resize(mask_window, (512, 512))
    
    masked = cv2.bitwise_or(mask_wall, mask_window)
    masked = np.uint8(cv2.bitwise_or(masked, mask_door))
    
    masked[masked==255] = 1
    masked[masked==160] = 2
    masked[masked==80 ] = 3
    masked[masked > 3 ] = 0
    
    im = Image.fromarray(masked).resize(size)
    im.save(f'F:/Crimea/submit/{name}')

100%|██████████| 1500/1500 [11:14<00:00,  2.22it/s]
